In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import C14preproc as prep
import C14utils
from mpl_toolkits.basemap import Basemap, cm
from netCDF4 import Dataset as NetCDFFile
from scipy.interpolate import interp1d
import auxiliary_lib as au
import isamcalc_lib as isam
import socplot_lib as socplt
import os.path
from os import path


In [2]:
# Get NCSCD 30cm and 100cm SOC 
fncscd30 = '/data/jain1/c/sshu3/SBGC/data/NCSCDv2_Circumpolar_netCDF_05deg/NCSCDv2_Circumpolar_WGS84_SOCC30_05deg.nc' 
fncscd70 = '/data/jain1/c/sshu3/SBGC/data/NCSCDv2_Circumpolar_netCDF_05deg/NCSCDv2_Circumpolar_WGS84_SOCC70_05deg.nc'
nclu_30=NetCDFFile(fncscd30,'r')
soc30 = nclu_30.variables['NCSCDv2'][:]
soc30 = soc30 / 10.
soc30 = np.flipud(soc30)
latnc = nclu_30.variables['lat'][:]
lonnc = nclu_30.variables['lon'][:]
latnc = np.flipud(latnc)

nclu_70=NetCDFFile(fncscd70,'r')
soc70 = nclu_70.variables['NCSCDv2'][:]
soc70 = soc70 / 10.
soc70 = np.flipud(soc70)

# Permafrost mask
fpfmask = '/data/jain1/c/sshu3/SBGC/data/NCSCDv2_Circumpolar_netCDF_05deg/ncscd_masked_0.5x0.5.nc'
nclu_mask = NetCDFFile(fpfmask,'r')
pfmask = nclu_mask.variables['PFMASK'][249:360,:]
latmask = nclu_mask.variables['lat'][249:360]
lonmask = nclu_mask.variables['lon'][:]

# Mask out the values outside the permafrost
soc30[pfmask<=0] = float('nan')
soc30[soc30<=0] = float('nan')
soc30[soc30>=500] = float('nan')
soc70[pfmask<=0] = float('nan')
soc70[soc70<=0] = float('nan')
soc70[soc70>=500] = float('nan')


/data/keeling/a/sshu3/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:25: MaskedArrayFutureWarning: setting an item on a masked array which has a shared mask will not copy the mask and also change the original mask array in the future.
Check the NumPy 1.11 release notes for more information.
/data/keeling/a/sshu3/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:26: RuntimeWarning: invalid value encountered in less_equal
/data/keeling/a/sshu3/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:27: RuntimeWarning: invalid value encountered in greater_equal
/data/keeling/a/sshu3/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:28: MaskedArrayFutureWarning: setting an item on a masked array which has a shared mask will not copy the mask and also change the original mask array in the future.
Check the NumPy 1.11 release notes for more information.
/data/keeling/a/sshu3/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:29: RuntimeWarning: i

In [ ]:
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
lon,lat = np.meshgrid(lonnc,latnc) #Returns coordinate matrices from coordinate vectors
x,y = map(lon,lat)
cs = map.pcolormesh(x,y,soc70[:,:],
              norm=colors.Normalize(vmin=0,vmax=300.),cmap=plt.cm.jet) # use log color scale,
plt.show()

In [6]:
np.nanmean(soc30)

11.506725599198031

In [ ]:
#================================================================================
# Initilize the site information
#================================================================================
# obs
obs_fname = 'Non_peat_data_permafrost.csv'
data = pd.read_csv(obs_fname,encoding='iso-8859-1',index_col='ProfileID', skiprows=[1])
all_profid = data.index.unique()
lons = prep.getvarxls(data,'Lon',all_profid[0:16],0)
lats = prep.getvarxls(data,'Lat',all_profid[0:16],0)

# Calibration
isam_cal_d14c_fname = 'isam_dc14_cali.dat'
isam_cal_soc_fname = 'isam_soc_cali.dat'

# Um's results
isam_um_d14c_fname = 'isam_um_dc14.dat'
isam_um_soc_fname = 'isam_um_soc.dat'

site = {
    "43"  : 0,
    "110" : 1,
    "143" : 2,
    "197" : 3,
    "146" : 4,   
}

yr_dict = {
    "43"  : 2001,
    "110" : 2005,
    "143" : 2007,
    "197" : 1999,
    "146" : 2002,   
}



In [ ]:
# Initialization
d14cm_test = [None] * 5
soccm_test = [None] * 5
bulk_d14c_topsoil = np.zeros((18, 5))
tau_topsoil = np.zeros((18, 5))
bulk_d14c_subsoil = np.zeros((18, 5))
tau_subsoil = np.zeros((18, 5))
    
for siteid, ind in site.items():
    # Read in D14C of the ISAM model output
    fname = 'isam_dc14_'+str(siteid)+'sen.dat'
    d14cm_test[ind] = pd.read_table(fname, header=None, delimiter=r"\s+")
    d14cm_test[ind].columns = ['ID', 'Layer1', 'Layer2', 'Layer3', 'Layer4', 'Layer5', 'Layer6', 'Layer7', 'Layer8', 'Layer9', 'Layer10']
    d14cm_test[ind] = d14cm_test[ind].set_index('ID')
    # Read in SOC
    fname = 'isam_soc_'+str(siteid)+'sen.dat'
    soccm_test[ind] = pd.read_table(fname, header=None, delimiter=r"\s+")
    soccm_test[ind].columns = ['ID', 'Layer1', 'Layer2', 'Layer3', 'Layer4', 'Layer5', 'Layer6', 'Layer7', 'Layer8', 'Layer9', 'Layer10']
    soccm_test[ind] = soccm_test[ind].set_index('ID')

    # Weighted C14 for topsoil and subsoil separatedly
    # Totally 18 cases for each site
    sampleyr = yr_dict[str(siteid)] * np.ones(len(d14cm_test[ind]))
    for i in np.arange(0,18):
        bulk_d14c_topsoil[i,ind] = isam.avg_wt_topsoil(soccm_test[ind].as_matrix()[i,], d14cm_test[ind].as_matrix()[i,])
        bulk_d14c_subsoil[i,ind] = isam.avg_wt_subsoil(soccm_test[ind].as_matrix()[i,], d14cm_test[ind].as_matrix()[i,])

    # Calculate the turnover time
    tau, cost = C14utils.cal_tau(bulk_d14c_topsoil[:,ind], sampleyr, 1, 0)
    tau_topsoil[:, ind] = tau[:,0]
    tau, cost = C14utils.cal_tau(bulk_d14c_subsoil[:,ind], sampleyr, 1, 0)
    tau_subsoil[:, ind] = tau[:,0]

    #tau[tau==2.00000000e+03] = np.float("nan")
    #data['tau'] = pd.Series(tau[:,0], index=data.index)
    #data.nodedepth = z*100
    #ttt=tau.reshape(255)


In [ ]:
a='stt'
b='ear'
c='pdo'
d='.nc'
"%s/%s_%s.nc"% (a,b,c)